# Détection des Violations RGPD dans les Données des Acteurs LVAO

Ce notebook a pour objectif de détecter les violations du RGPD dans les données d'acteurs de LVAO et de mettre à jour la table PostgreSQL `qfdmo_revisionacteur` en conséquence.

## Sources des Données

Les données utilisées dans ce notebook proviennent de deux sources principales :

### Sources Sirene

La base Sirene des entreprises est accessible via [data.gouv.fr](https://www.data.gouv.fr/fr/datasets/base-sirene-des-entreprises-et-de-leurs-etablissements-siren-siret/). Les fichiers suivants sont utilisés :

- `StockUniteLegale_utf8` : Contient les informations légales sur les unités légales.
- `StockEtablissement_utf8` : Contient les informations sur les établissements.

### Sources LVAO

Les données d'acteurs LVAO sont extraites de la table suivante :

- `qfdmo_finalacteur` : Table contenant les données finales des acteurs.

## Objectif

L'objectif de ce notebook est de réaliser les opérations suivantes :

1. Charger et Enrichir les données d'acteurs des données provenant des sources Sirene.
2. Filter sur label_reparacteur==False & statut=='ACTIF' & 'source_id'==4
3. Identifier les éventuelles violations du RGPD dans ces données
4. Identifier l'existence d'un nom légal alternatif
5. Générer un fichier CSV avec le schéma de la table `qfdmo_revisionacteur`.


Télécharger la derniere version de StockUniteLegale et StockEtablissement sur [data.gouv.fr](https://www.data.gouv.fr/fr/datasets/base-sirene-des-entreprises-et-de-leurs-etablissements-siren-siret/)

In [29]:
stock_unite_legale = "/Users/hamzaa/data/quefairedemesobjets/StockUniteLegale_utf8.csv"
stock_etablissement = "/Users/hamzaa/data/quefairedemesobjets/StockEtablissement_utf8.csv"

In [30]:
!pip install ratelimiter


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [31]:
import pandas as pd
import re
from sqlalchemy import create_engine
from decouple import config


In [32]:
from ratelimiter import RateLimiter

@RateLimiter(max_calls=7, period=1)
def call_annuaire_entreprises(siren):
    params = {
        'q': siren
    }
    base_url = 'https://recherche-entreprises.api.gouv.fr'
    endpoint = '/search'

    try:
        response = requests.get(url=f'{base_url}{endpoint}', params=params)

        if response.status_code == 200:
            data = response.json()
            # Vérifiez si 'results' existe et contient au moins un élément
            if data.get('results') and len(data['results']) > 0:
                dirigeants = data['results'][0].get('dirigeants', [])
                noms = [dir['nom'] for dir in dirigeants if 'nom' in dir and dir.get('type_dirigeant') == 'personne physique']
                return noms
            else:
                return []
        else:
            return []
    except requests.exceptions.RequestException as e:
        # Gérer les exceptions liées aux requêtes
        print(f'Une erreur est survenue lors de la requête: {e}')
        return ['Une erreur de requête est survenue']

def check_rgpd_violation(row):
    # Check for non-null values first to avoid errors
    if pd.notnull(row['nomUniteLegale']) and pd.notnull(row['nom']) and pd.notnull(row['statutDiffusionUniteLegale']):
        names_to_check = [row['nomUniteLegale']]
        if pd.notnull(row['nomUsageUniteLegale']):
            names_to_check.append(row['nomUsageUniteLegale'])
        
        pattern = r'\b(' + r'|'.join(re.escape(name) for name in names_to_check) + r')\b'
        name_match = bool(re.search(pattern, row['nom'], re.IGNORECASE))
        return name_match 
    else:
        return False



def check_rgpd_violation_using_annuaire_entreprise(row):
    names_to_check = call_annuaire_entreprises(row['siren'])
    if pd.notnull(row['nom']) and names_to_check:
        pattern = r'\b(' + r'|'.join(re.escape(name) for name in names_to_check) + r')\b'
        name_match = bool(re.search(pattern, row['nom'], re.IGNORECASE))
        return name_match 
    else:
        return False


def split_dataframe_into_batches(df, batch_size=10000):
    num_rows = df.shape[0]
    num_batches = (num_rows + batch_size - 1) // batch_size  # Calculate the number of batches needed
    batches = [df[i*batch_size:(i+1)*batch_size] for i in range(num_batches)]
    return batches



def check_alternative_legal_name(row):
    old_state = row['rgpd_violation']
    if pd.notnull(row['denominationUsuelle1UniteLegale']) and pd.notnull(row['nomUniteLegale']) and pd.notnull(row['statutDiffusionUniteLegale']):
        pattern2 =  r'\b' + re.escape(str(row['denominationUsuelle1UniteLegale'])) + r'\b'
        name_match = bool(re.search(pattern2, row['nomUniteLegale'], re.IGNORECASE))
        if old_state==True:
            if name_match==False:
                return name_match
        return old_state
    else:
        return old_state

In [33]:
# create prod engine
connection_string_prod = f'postgresql://{user}:{password}@{host}:{port}/{db_name}'
engine_prod = create_engine(connection_string_prod)

In [34]:
#load sources
df_unite_legale = pd.read_csv(stock_unite_legale)
df_etab = pd.read_csv(stock_etablissement)
df_final_acteur = pd.read_sql_table('qfdmo_finalacteur', engine_prod)


/var/folders/0b/ssm8dl5n5td_t_2lb_8qn6500000gn/T/ipykernel_5995/2746981125.py:2: DtypeWarning: Columns (2,4,5,6,7,8,9,10,11,12,13,17,21,22,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_unite_legale = pd.read_csv(stock_unite_legale)
/var/folders/0b/ssm8dl5n5td_t_2lb_8qn6500000gn/T/ipykernel_5995/2746981125.py:3: DtypeWarning: Columns (5,7,12,16,18,20,21,24,25,26,27,28,29,30,31,33,34,35,36,38,42,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_etab = pd.read_csv(stock_etablissement)
/Users/hamzaa/data/quefairedemesobjets/venv/lib/python3.9/site-packages/pandas/io/sql.py:1665: SAWarning: Did not recognize type 'geometry' of column 'location'
  self.meta.reflect(bind=self.con, only=[table_name], views=True)


In [35]:
df_filtered = df_final_acteur[(df_final_acteur['label_reparacteur']==False) & (df_final_acteur['statut']=='ACTIF') & (df_final_acteur['source_id']==4) ]

In [36]:
df_filtered['siret'] = df_filtered['siret'].astype(str)
df_etab['siret'] = df_etab['siret'].astype(str)

/var/folders/0b/ssm8dl5n5td_t_2lb_8qn6500000gn/T/ipykernel_5995/1345952353.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['siret'] = df_filtered['siret'].astype(str)


In [37]:
df_merge = pd.merge(df_filtered, df_etab[['siren','siret']],on=['siret'])

In [38]:
df_sirene = pd.merge(df_merge,df_unite_legale[['siren','denominationUsuelle1UniteLegale','statutDiffusionUniteLegale',
       'prenom1UniteLegale', 'prenom2UniteLegale', 'prenom3UniteLegale',
       'prenom4UniteLegale', 'prenomUsuelUniteLegale', 
       'nomUniteLegale', 'nomUsageUniteLegale',
       'caractereEmployeurUniteLegale']], on = ['siren'])

In [39]:
# Apply the function across the rows to check for RGPD violations
df_sirene['rgpd_violation'] = df_sirene.apply(check_rgpd_violation, axis=1)
df_sirene['check_alternative_legal_name'] = df_sirene.apply(check_alternative_legal_name, axis=1)
df_sirene.loc[(df_sirene['rgpd_violation']==True) & (df_sirene['check_alternative_legal_name']==False), 'nom'] = df_sirene['denominationUsuelle1UniteLegale']
df_sirene.loc[(df_sirene['rgpd_violation'] == True) & (df_sirene['check_alternative_legal_name'] == False), 'name_change'] = True
df_sirene.loc[(df_sirene['rgpd_violation'] == True) & (df_sirene['check_alternative_legal_name'] == False), 'rgpd_violation'] = False
df_sirene = df_sirene[df_sirene['statutDiffusionUniteLegale'] != 'P']

/var/folders/0b/ssm8dl5n5td_t_2lb_8qn6500000gn/T/ipykernel_5995/795525349.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sirene.loc[(df_sirene['rgpd_violation'] == True) & (df_sirene['check_alternative_legal_name'] == False), 'name_change'] = True


In [40]:
cols = df_final_acteur.columns.tolist()
df_sirene.loc[(df_sirene['rgpd_violation']==True),'statut']="SUPPRIME"

### Annuaire entreprise RGPD check

In [42]:
actors_to_check = df_sirene[df_sirene['nomUniteLegale'].isnull()]

actors_to_check['rgpd_violation_ae'] = actors_to_check.apply(check_rgpd_violation_using_annuaire_entreprise, axis=1)


/var/folders/0b/ssm8dl5n5td_t_2lb_8qn6500000gn/T/ipykernel_5995/675051470.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actors_to_check['rgpd_violation_ae'] = actors_to_check.apply(check_rgpd_violation_using_annuaire_entreprise, axis=1)


In [50]:
actors_to_check.to_csv('rpg_violation_annuaire_entreprise_25032024.csv')

In [ ]:
df_sirene[cols].loc[(df_sirene['rgpd_violation']==True)].to_csv('rgpd_violation_lvao_actors_13032024.csv')
df_sirene[cols].loc[(df_sirene['rgpd_violation']==True)]

### Write to PostgreSql to revision acteur table

In [56]:
import psycopg2
from psycopg2 import sql
# Access variables in .env


In [ ]:

conn = psycopg2.connect(
    dbname=db_name, 
    user=user, 
    password=password, 
    host=host,
    port=port
)
conn.autocommit = True
cursor = conn.cursor()

for index, row in df_sirene.loc[df_sirene['rgpd_violation'] == True].iterrows():
    query = sql.SQL("UPDATE qfdmo_revisionacteur SET nom = %s, statut = %s WHERE identifiant_unique = %s;")
    cursor.execute(query, (row['nom'], row['statut'], row['identifiant_unique']))

cursor.close()
conn.close()

In [55]:

conn = psycopg2.connect(
    dbname=db_name, 
    user=user, 
    password=password, 
    host=host,
    port=port
)
conn.autocommit = True
cursor = conn.cursor()

for index, row in actors_to_check.loc[actors_to_check['rgpd_violation_ae'] == True].iterrows():
    query = sql.SQL("UPDATE qfdmo_revisionacteur SET statut = %s WHERE identifiant_unique = %s;")
    cursor.execute(query, (row['statut'], row['identifiant_unique']))

cursor.close()
conn.close()